<div style="text-align: center">
<h1><font color="black" size=8>PROYECTO DE CONSOLIDACIÓN</font></h1>

<div style="text-align: left">
<h1><font color="#009D7F" size=5>Alejandro Santiago Bitria</font></h1>


## Reto 1: Capturador de definiciones 📙

En la sesión de APIs hemos visto cómo utilizar la [API de Oxford Dictionaries](https://developer.oxforddictionaries.com/) para capturar definiciones dada una palabra.

En la sesión de Web Scraping hemos visto cómo capturar elementos concretos de una página Web gracias a Selenium y su selección de elementos a través de la directiva `.find_element_by_xpath("...")`

**Objetivo**: El reto consiste en <u>conectar dos sistemas distintos</u> a través de las dos técnicas mencionadas anteriormente:

1. Accede a la página https://www.palabrasaleatorias.com/ y captura **10 palabras aleatorias** en español con las técnicas vistas de Web Scraping `.find_element_by_xpath("...)"`

2. Utiliza la API de Oxford Dictionaries para obtener sus definiciones. Recuerda que necesitas [registrarte en la página](https://developer.oxforddictionaries.com/) y obtener tus credenciales `app_id` y `app_key` para consultar la API.

A continuación tienes la lista de recursos.  *Recuerda que puedes consultar las presentaciones y ejercicios de las sesiones ya realizadas.*

**Recursos:**
 - **Página palabras aleatorias:** https://www.palabrasaleatorias.com/
 - **Obtención API Oxford Dictionaries:** https://developer.oxforddictionaries.com/login
 - **Cómo utilizar la API de Oxford Dictionaries:** https://developer.oxforddictionaries.com/login
    

In [484]:
api_oxford_app_id  = "4d80819c"
api_oxford_app_key = "f54848e954c03549230b53cd1f5cf029"
endpoint = "entries"
language_code = "es-es"
#Resto de variables, comprobad la documentación de Oxford Dictionaries

In [591]:
import json
import requests
from selenium import webdriver
from time import sleep

In [592]:
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("-private")

driver = webdriver.Firefox(options=firefox_options)
driver.get("https://www.palabrasaleatorias.com")

In [593]:
#Acepta cookies
sleep(2)
cookies = driver.find_element_by_xpath("/html/body/div/div[2]/div[1]/div[2]/div[2]/button[1]/p").click()

#Selecciona 10 en el desplegable
sleep(2)
selector = driver.find_element_by_xpath("/html/body/center/center/table[1]/tbody/tr/td/form/select[1]").send_keys("10")

#Haz click en Generar palabras
sleep(2)
generar_palabras = driver.find_element_by_xpath("/html/body/center/center/table[1]/tbody/tr/td/form/input").click()
sleep(5)

In [629]:
#Captura de todas las palabras ya generadas

for i in range (1, 11):
    selector_xpath = "/html/body/center/center/table[1]/tbody/tr/td/div[" + str(i) + "]" 
    #print(selector_xpath)
    #print(driver.find_element_by_xpath(selector_xpath).text)
    word_id = driver.find_element_by_xpath(selector_xpath).text
    url = "https://od-api.oxforddictionaries.com/api/v2/" + endpoint + "/" + language_code + "/" + word_id.lower()
    print(url)
    r = requests.get(url, headers = {"app_id": api_oxford_app_id, "app_key": api_oxford_app_key})
    print(r.status_code)

https://od-api.oxforddictionaries.com/api/v2/entries/es/convidar
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/hoz
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/sintonizar
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/frutilla
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/resultado
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/boleto
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/tracción
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/coletas
404
https://od-api.oxforddictionaries.com/api/v2/entries/es/artista
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/vapor
200


Observamos que uno de las 10 palabras (coletas) **da error** por motivos que desconozco.

Vamos a intentar guardar las definiciones en un Dataframe

In [642]:
import pandas as pd

diccionario = pd.DataFrame(columns=['Definiciones'])
diccionario.drop(diccionario.index, inplace=True)
diccionario

,Definiciones


In [643]:
try:  
    
    for i in range (1, 11):
        selector_xpath = "/html/body/center/center/table[1]/tbody/tr/td/div[" + str(i) + "]" 
    #print(selector_xpath)
    #print(driver.find_element_by_xpath(selector_xpath).text)
        word_id = driver.find_element_by_xpath(selector_xpath).text
        url = "https://od-api.oxforddictionaries.com/api/v2/" + endpoint + "/" + language_code + "/" + word_id.lower()
        print(url)
        r = requests.get(url, headers = {"app_id": api_oxford_app_id, "app_key": api_oxford_app_key})
        print(r.status_code)
        oxford = json.loads(r.text)

        for i in oxford["results"]:
            for j in i["lexicalEntries"]:
                for k in j["entries"]:
                    for v in k["senses"]:
                        diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True) 

except KeyError:
    pass

https://od-api.oxforddictionaries.com/api/v2/entries/es/convidar
200
https://od-api.oxforddictionaries.com/api/v2/entries/es/hoz


C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\305141

200
https://od-api.oxforddictionaries.com/api/v2/entries/es/sintonizar


C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)


200
https://od-api.oxforddictionaries.com/api/v2/entries/es/frutilla


...

200
https://od-api.oxforddictionaries.com/api/v2/entries/es/tracción


C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\305141

200
https://od-api.oxforddictionaries.com/api/v2/entries/es/coletas


C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\3051415602.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diccionario = diccionario.append({"Definiciones": v["definitions"], "Termino": oxford["id"]}, ignore_index=True)


404


Tras varios intentos, no consigo que tras el error siga con el resto de definiciones por lo que se corta ahí la búsqueda. Al menos con el pass no salta el KeyError y queda algo más limpio el código.

In [644]:
diccionario

,Definiciones,Termino
0,[Pedir a una persona que participe en una cele...,convidar
1,[Pagar a una persona algo que le resulta agrad...,convidar
2,[Ofrecer a una persona que haga algo que le in...,convidar
3,[Animar o estimular a una persona a hacer lo q...,convidar
4,"[Invitar a una persona con algo, especialmente...",convidar
5,[Ofrecerse voluntariamente para algo],convidar
6,[Apero de labranza que sirve para segar; está ...,hoz
7,"[Paso entre montañas profundo, estrecho y de p...",hoz
8,[Poner un aparato receptor de ondas en sintoní...,sintonizar
9,[Ajustar convenientemente los canales de una r...,sintonizar


Finalmente lo guardamos en un **formato csv**.

In [645]:
diccionario.to_csv("Diccionario.csv", header = ['Definiciones', 'Términos'])

# Reto 2: La base de datos real, pero falsa 👨👩

Todo proyecto necesita un conjunto de datos de prueba. No siempre tenemos oportunidad de disponer información real asi que utilizamos información aleatoria. Lo importante de esos datos es que sean de calidad, que los tipos de datos a generar estén dentro del espectro de información esperado.

Es por ello que en Internet existen múltiples páginas web que ofrecen información aleatoria muy diversa de forma gratuita y gracias a Web Scraping podemos crear de bases de datos de usuarios con muy poco esfuerzo. Unos ejemplos:
- https://namefake.com/
- https://es.fakenamegenerator.com/
- https://www.fakepersongenerator.com/
- http://www.genware.es/


**Objetivo**: El siguiente reto consiste en construir una base de datos de  **al menos 50 personas** físicas. La base de datos debe contener al menos la siguiente información:
- Nombre completo y apellidos
- Dirección física
- Número de telefóno móvil
- Tarjeta de crédito (Discount number)
- Fecha Expiración

Toda esta información se puede obtener utilizando el primer ejemplo: https://namefake.com/

Una vez construida la base de datos, <u>supongamos que trabajamos en una entidad  de tarjetas monedero (prepago) y queremos crear una campaña de marketing de retención de clientes dado que las tarjetas cuando expiran no se renuevan automáticamente.</u>

Muestra una tabla con todos los clientes que esté ordenada según la proximidad a la fecha de caducidad de las tarjetas.




**Extra**: *En ocasiones no podemos obtener toda la información de una sola página y necesitamos unir informacion de varios generadores. Extiende la base de datos actual con la información de https://es.fakenamegenerator.com/ incluyendo el elemento de "Coordenadas geográficas"*

In [374]:
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep

In [391]:
#Abrimos navegador Firefox
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("-private")
driver = webdriver.Firefox(options=firefox_options)

In [411]:
#Consultamos página web
driver.get("https://es.fakenamegenerator.com/gen-random-sp-sp.php")

In [412]:
nombre_completo = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[1]/h3').text
direccion_fisica = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[1]/div').text.replace("\n",", ")
telefono_movil = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[2]/dl[2]/dd').text
tarjeta_credito = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[2]/dl[12]/dd').text
fecha_expiracion = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[2]/dl[13]/dd').text

Tal como pedía el extra, añadimos las **coordenadas geográficas** de la otra página web:

In [413]:
driver.get("https://namefake.com/")

In [429]:
coordenadas_geograficas = driver.find_element_by_xpath("/html/body/div[3]/div/div[1]/div[1]").text

In [430]:
print(nombre_completo)
print(direccion_fisica)
print(telefono_movil)
print(tarjeta_credito)
print(fecha_expiracion)
print(coordenadas_geograficas)

Caleb Irizarry Trejo
San Andrés, 90, 37255 Barruecopardo
768 961 277
5448 9697 1334 6687
6/2025
Geo coordinates
-42.657687,-7.898408


In [416]:
import pandas as pd

df = pd.DataFrame(columns=['nombre','apellidos','direccion','telefono','tarjeta_num','tarjeta_exp_mes','tarjeta_exp_anyo', 'coordenadas'])
df

,nombre,apellidos,direccion,telefono,tarjeta_num,tarjeta_exp_mes,tarjeta_exp_anyo,coordenadas


In [432]:
#Formato de variables
nombre = nombre_completo.split()[0]
apellidos = ' '.join(nombre_completo.split()[1:])

tarjeta_exp_mes = int(fecha_expiracion.split("/")[0])
tarjeta_exp_anyo = int(fecha_expiracion.split("/")[1])
coordenadas = coordenadas_geograficas.split()[2]

In [433]:
#Vaciamos dataframe
df.drop(df.index, inplace=True)

#Añadimos datos al dataframe
df = df.append({"nombre": nombre, "apellidos": apellidos, "direccion": direccion_fisica, "telefono": telefono_movil, "tarjeta_num": tarjeta_credito, "tarjeta_exp_mes": tarjeta_exp_mes, "tarjeta_exp_anyo": tarjeta_exp_anyo, "coordenadas": coordenadas},
                    ignore_index=True)
df

C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\92170070.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"nombre": nombre, "apellidos": apellidos, "direccion": direccion_fisica, "telefono": telefono_movil, "tarjeta_num": tarjeta_credito, "tarjeta_exp_mes": tarjeta_exp_mes, "tarjeta_exp_anyo": tarjeta_exp_anyo, "coordenadas": coordenadas},


,nombre,apellidos,direccion,telefono,tarjeta_num,tarjeta_exp_mes,tarjeta_exp_anyo,coordenadas
0,Caleb,Irizarry Trejo,"San Andrés, 90, 37255 Barruecopardo",768 961 277,5448 9697 1334 6687,6,2025,"-42.657687,-7.898408"


In [436]:
df.drop(df.index, inplace=True)

In [440]:
for i in range(0,50):
    driver.get("https://es.fakenamegenerator.com/gen-random-sp-sp.php")
    #Pausa para no ser tan "robotizado"
    sleep(2)
    #Capturamos información
    nombre_completo = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[1]/h3').text
    direccion_fisica = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[1]/div').text.replace("\n",", ")
    telefono_movil = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[2]/dl[2]/dd').text
    tarjeta_credito = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[2]/dl[12]/dd').text
    fecha_expiracion = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/div[3]/div[2]/div[2]/div/div[2]/dl[13]/dd').text
    
    
    driver.get("https://namefake.com/")
    coordenadas_geograficas = driver.find_element_by_xpath("/html/body/div[3]/div/div[1]/div[1]").text
    sleep(2)
    
    #Formato de variables
    nombre = nombre_completo.split()[0]
    apellidos = ' '.join(nombre_completo.split()[1:])

    tarjeta_exp_mes = int(fecha_expiracion.split("/")[0])
    tarjeta_exp_anyo = int(fecha_expiracion.split("/")[1])
    coordenadas = coordenadas_geograficas.split()[2]


    #Incluimos datos en tabla
    df = df.append({"nombre": nombre, "apellidos": apellidos, "direccion": direccion_fisica, "telefono": telefono_movil, "tarjeta_num": tarjeta_credito, "tarjeta_exp_mes": tarjeta_exp_mes, "tarjeta_exp_anyo": tarjeta_exp_anyo, "coordenadas": coordenadas},
                    ignore_index=True)



C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\2913033376.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"nombre": nombre, "apellidos": apellidos, "direccion": direccion_fisica, "telefono": telefono_movil, "tarjeta_num": tarjeta_credito, "tarjeta_exp_mes": tarjeta_exp_mes, "tarjeta_exp_anyo": tarjeta_exp_anyo, "coordenadas": coordenadas},
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\2913033376.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"nombre": nombre, "apellidos": apellidos, "direccion": direccion_fisica, "telefono": telefono_movil, "tarjeta_num": tarjeta_credito, "tarjeta_exp_mes": tarjeta_exp_mes, "tarjeta_exp_anyo": tarjeta_exp_anyo, "coordenadas": coordenadas},
C:\Users\ALEX\AppData\Local\Temp\ipykernel_15128\2913033376.py:27: FutureWarning: The frame.

In [441]:
df.sort_values(by=["tarjeta_exp_anyo", "tarjeta_exp_mes"])

,nombre,apellidos,direccion,telefono,tarjeta_num,tarjeta_exp_mes,tarjeta_exp_anyo,coordenadas
17,Daira,Nazario Palomino,"Outid de Arriba, 59, 43594 El Pinell de Brai",777 662 549,5498 7238 4783 0895,2,2023,"-46.17038,-130.691934"
4,Amatista,Rojo Franco,"La Fontanilla, 1, 14206 Valsequillo",613 632 823,5314 7127 4382 5697,3,2023,"45.465673,168.004922"
33,Erico,Guerra Alfaro,"Ventanilla de Beas, 12, 28150 Valdetorres de J...",721 274 801,4716 0919 5271 2673,3,2023,"42.78824,160.723945"
32,Illed,Argüello Zúñiga,"Salzillo, 6, 32550 Viana do Bolo",602 434 050,5467 2609 7946 4851,5,2023,"67.339559,174.696442"
22,Querubín,Olvera Gallardo,"Ctra. de Fuentenueva, 48, 28300 Aranjuez",681 118 144,4929 8808 5985 2092,12,2023,"-11.821114,-43.819792"
20,Fiona,Alva Espino,"C/ Cuesta del Álamo, 70, 29679 Benahavís",603 935 873,5570 6470 6475 0091,1,2024,"-29.827543,19.908872"
46,Lorenz,Baeza Suárez,"C/ Cañada del Rosal, 86, 21580 Cabezas Rubias",614 590 236,5387 4625 2898 5603,1,2024,"54.131788,-154.870547"
16,Lupo,Hidalgo Ávalos,"Carretera del Muelle, 20, 12413 Castellnovo",648 320 797,5537 2826 1829 2499,2,2024,"-68.839149,-129.417025"
2,Fabia,Pizarro Reyes,"Zumalakarregi etorbidea, 35, 07315 Mancor de l...",672 421 637,5404 1122 4303 4885,4,2024,"59.896593,-29.308663"
30,Henrey,Lucero Nieto,"Avda. Explanada Barnuevo, 24, 35290 San Bartol...",643 718 921,5209 3218 2600 4820,4,2024,"66.35952,-168.137555"


Y con ello ya tendríamos el perfil de las **50 personas** con las diferentes características específicas.

# Reto 3: El comparador de precios 📊📉

Los comparadores de precios <u>es uno de los nichos de mercado más lucrativos en Internet.</u> Dado un producto o servicio a nivel de usuario nos beneficiamos del precio más bajo existente y a nivel de empresa éstas obtienen grandes beneficios gracias a sistemas de referidos o de comisiones por venta realizada.

<u>En esta ocasión vamos a realizar un pequeño comparador de precios basado en el ISBN.</u> Un ISBN es un código normalizado internacional para libros (International Standard Book Number). Los ISBN tuvieron 10 dígitos hasta diciembre de 2006 pero, desde enero de 2007, tienen siempre 13 dígitos. 


Por ejemplo, el ISBN [9788478884452](https://www.google.es/search?q=9788478884452) (haz click) corresponde al libro "Harry Potter y la piedra filosofal".

Dado que no disponemos de acceso a APIs de las tiendas principales <u>capturaremos el precio a través de Web Sraping.</u>

**Objetivo:** El objetivo es realizar la consulta del ISBN de Harry Potter y la Piedra filosofal" en al menos tres de las siguientes tiendas propuestas. Captura el primer resultado en EUROS, y devolver cómo resultado cuál tiene el precio más bajo. Como punto extra, incluye posteriormente cuál es la diferencia de precio en euros y en porcentaje  respecto al valor más alto detectado para saber cuánto nos estamos ahorrando.

- https://www.casadellibro.com/
- https://www.libreriacentral.com/
- https://www.iberlibro.com/
- https://www.amazon.es/
- https://ebay.es
- https://www.elcorteingles.es/

Recuerda que para cada página debes realizar ingeniería inversa, averiguando cómo se comportan las URLs de cada sitio web para hacer una búsqueda directa.


**Extra**: *¿Conoces alguna otra página donde comprar libros? Inclúyela en el comprador*

**Extra 2**: *¿Cómo podríamos obtener una evolución del precio durante una semana?*

In [238]:
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep

De todas las opciones de libros, optamos por el mencionado en el enunciado de **Harry Potter y la piedra filosofal**

In [239]:
ISBN_LIBRO = "9788478884452" # Harry Potter y la piedra filosofal
debug = True

In [250]:
#Abrimos navegador Firefox
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("-private")
driver = webdriver.Firefox(options=firefox_options)

Ahora iremos por las tiendas descritas en el enunciado (luego buscaremos más):

In [294]:
#Tienda 1 - Casa del libro
driver.get("https://www.casadellibro.com/?q=" + ISBN_LIBRO)
sleep(7)
try:
    casadellibro = driver.find_element_by_xpath('//*[@id="ebx-grid"]/article/div/div/div/div/div/span[2]')
    precio_casadellibro = casadellibro.text
except:
    precio_casadellibro = "99999.99"
if debug:
    print(precio_casadellibro)

15.2 €


In [295]:
#Tienda 2 - Librería Central
driver.get("https://www.libreriacentral.com/SearchResults.aspx?st=" + ISBN_LIBRO + "&cId=0&sm=qck")
try:
    libreriacentral = driver.find_element_by_xpath('//*[@id="ctl00_CPHMainCenter_searchedProducts_rptrProdSearched_ctl00_ProductSmall_lblPriceWeb"]')
    precio_libreriacentral = libreriacentral.text
except:
    precio_libreriacentral = "99999.99"
if debug:
    print(precio_libreriacentral)

14,25


In [296]:
#Tienda 3 - IberLibro 
driver.get("https://www.iberlibro.com/servlet/SearchResults?cm_sp=sort-_-SRP-_-Results&kn="+ ISBN_LIBRO + "&n=100121501&sortby=2")
try: 
    iberlibro = driver.find_element_by_xpath('//*[@id="item-price-1"]')
    precio_iberlibro = iberlibro.text
except:
    precio_iberlibro = "EUR 99999.99"
if debug:
    print(precio_iberlibro)

EUR 14,25


In [297]:
#Tienda 4 - Amazon
driver.get("https://www.amazon.es/s?k=" + ISBN_LIBRO + "&__mk_es_ES=ÅMÅŽÕÑ&crid=2XJPC4J3ZO6QY&sprefix=9788478884452%2Caps%2C80&ref=nb_sb_noss")

try: 
    amazon = driver.find_element_by_xpath('//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[3]/div/div/div/div/div[2]/div[3]/div[2]/a/span[2]/span[2]/span[1]')
    precio_amazon = amazon.text
except:
    precio_amazon = "EUR 99999.99"
if debug:
    print(precio_amazon)

15,19


In [298]:
#Tienda 5 - Ebay
driver.get("https://www.ebay.es/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=" + ISBN_LIBRO + "&_sacat=0")
try: 
    ebay = driver.find_element_by_xpath('//*[@id="srp-river-results"]/ul/li[2]/div/div[2]/div[3]/div[1]/span/span')
    precio_ebay = ebay.text
except:
    precio_ebay = "EUR 99999.99"
if debug:
    print(precio_ebay)

10,54 EUR


In [299]:
#Tienda 6 - Corte Inglés
driver.get("https://www.elcorteingles.es/libros/A1870800-harry-potter-y-la-piedra-filosofal-harry-potter-1-tapa-dura/?s=" + ISBN_LIBRO + "&hierarchy=&deep_search=&stype=text_box")
try: 
    corte_ingles = driver.find_element_by_xpath('//*[@id="product-detail-container"]/div[1]/div[2]/div/div[4]/div/p[1]')
    precio_corte_ingles = iberlibro.text
except:
    precio_corte_ingles = "EUR 99999.99"
if debug:
    print(precio_corte_ingles)

EUR 99999.99


Aunque el precio es de 14,25€ dicha página lo tiene capado y con selenium no se puede acceder via Firefox.

Añadimos **FNAC** y ya que soy de Zaragoza también la **Libreria París**.

In [300]:
#Tienda 7 - FNAC

driver.get("https://www.fnac.es/SearchResult/ResultList.aspx?SCat=0&Search=" + ISBN_LIBRO + "&sft=1&sa=0")
try: 
    fnac = driver.find_element_by_xpath('/html/body/div[3]/div/div[5]/div/article/form/div[3]/div/div/div/div[1]/strong')
    precio_fnac = fnac.text
except:
    precio_fnac = "EUR 99999.99"
if debug:
    print(precio_fnac)

14,25€


In [301]:
#Tienda 8 - Libreria París

driver.get("https://www.libreriaparis.com/libro/9624/harry-potter-y-piedra-filosofal-vol1-edicion-tapa-dura/" + ISBN_LIBRO)
try: 
    libreria_paris = driver.find_element_by_xpath('//*[@id="page-content-wrapper"]/div[3]/div/div[2]/div/div/div/div/div[2]/div[4]/div/div[1]/p')
    precio_libreria_paris = libreria_paris.text
except:
    precio_libreria_paris = "EUR 99999.99"
if debug:
    print(precio_libreria_paris)

16.00€


Bien, aunque ya tenemos todos los precios, observamos que entre ellos hay diferencias no numéricas: las comas, unos con el símbolo del euro (€), otros con la palabra EUR...

Por ello, vamos a limpiar estos datos de tal manera que al compararlos sean **todos iguales**

In [317]:
#Limpieza simbolo euro

valor_casadellibro = precio_casadellibro.split()[0]


print(valor_casadellibro)

15.2


Con el precio de Fnac y Libreria París el valor icono de euros está incluido dentro del propio string (no hay espacio como en el otro caso) asi que simplemente lo quitamos

In [318]:
characters = "€"

for x in range(len(characters)):
    valor_fnac = precio_fnac.replace(characters[x],"")
    valor_libreria_paris =precio_libreria_paris.replace(characters[x],"")

print(valor_fnac, valor_libreria_paris)

14,25 16.00


In [319]:
#Limpieza de coma

valor_libreriacentral = precio_libreriacentral.replace(",",".")
valor_iberlibro = precio_iberlibro.replace(",",".")
valor_amazon = precio_amazon.replace(",",".")
valor_ebay = precio_ebay.replace(",",".")
valor_fnac = valor_fnac.replace(",",".")


print(valor_libreriacentral, valor_iberlibro, valor_amazon, valor_ebay, valor_fnac)

14.25 EUR 14.25 15.19 10.54 EUR 14.25


In [320]:
#Limpieza palabra EUR

valor_iberlibro = precio_iberlibro.split()[1].replace(",",".")
valor_ebay = valor_ebay.split()[0].replace(",",".")

print(valor_iberlibro, valor_ebay)

14.25 10.54


In [321]:
#Devuelve el valor más bajo

if  (min(valor_casadellibro, valor_libreriacentral, valor_iberlibro, valor_amazon, valor_ebay, valor_fnac, valor_libreriacentral) == 99999.99):
    print("No se ha encontrado ningun resultado en las tiendas observadas")
else:
    print("El valor más barato del libro con ISBN " + ISBN_LIBRO + " es " + str(min(valor_casadellibro, valor_libreriacentral, valor_iberlibro, valor_amazon, valor_ebay, valor_fnac, valor_libreriacentral)))

El valor más barato del libro con ISBN 9788478884452 es 10.54


En conclusión, donde es más barato comprarlo es en **Ebay** (aunque no se tiene en cuenta los posibles gastos de envío).

Respecto a la **pregunta extra 2**, la única solución sería hacer un Web Scraping simultáneo cada día de las páginas que estemos interesados en sus precios (dentro del mismo día hacerlo X veces, cada hora por ejemplo) y hacer la diferencia. Es algo muy estático pero no podemos estar metidos dentro de la página constamente porque al final tienen sistemas para detectarlo por lo que a tiempo real es imposible.. 